<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/speedy-transformer/notebooks/5.0b-mb-gradient-boosting-retraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install catboost
# !pip install wandb
# !pip install gcsfs
# !pip install google-cloud-storage
# !pip install fastparquet

In [ ]:
import json
import os
import pickle
from pathlib import Path

import gcsfs
import google.auth
import numpy as np
import pandas as pd
import wandb
from catboost import CatBoostClassifier, Pool
from google.colab import auth, output
from sklearn.metrics import accuracy_score

In [ ]:
# connect to google cloud storage
auth.authenticate_user()
credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)
# fs = gcsfs.GCSFileSystem(project="thesis")
# fs_prefix = "gs://"


In [ ]:


# https://github.com/KarelZe/thesis/blob/main/notebooks/
# 3.0a-mb-explanatory_data_analysis.ipynb
features_categorical = [
    ("bin_root", 8667),
    ("bin_option_type", 2),
    ("bin_issue_type", 6),
]

features_classical = [
    "TRADE_PRICE",
    "bid_ex",
    "ask_ex",
    "BEST_ASK",
    "BEST_BID",
    "price_ex_lag",
    "price_ex_lead",
    "price_all_lag",
    "price_all_lead",
    "chg_ex_lead",
    "chg_ex_lag",
    "chg_all_lead",
    "chg_all_lag",
    "prox_ex",
    "prox_best",
]

features_size = [
    "bid_ask_size_ratio_ex",
    "rel_bid_size_ex",
    "rel_ask_size_ex",
    "TRADE_SIZE",
    "bid_size_ex",
    "ask_size_ex",
    "depth_ex",
]

features_classical_size = [
    *features_classical,
    *features_size,
]


In [ ]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"


In [ ]:
# see https://wandb.ai/fbv/thesis/runs/kwlaw02g/overview?workspace=user-karelze
run = wandb.init(project="thesis", entity="fbv")

dataset = "fbv/thesis/ise_log_standardized:v1"
artifact = run.use_artifact(dataset)
data_dir = artifact.download()

study = "fbv/thesis/xl3n4thc.optuna:v99"
artifact = run.use_artifact(study)
study_dir = artifact.download()


model = "xl3n4thc_CatBoostClassifier_default.cbm:v9"
model_name = model.split("/")[-1].split(":")[0]

artifact = run.use_artifact(model)
model_dir = artifact.download()


In [ ]:
model = CatBoostClassifier()
model.load_model(fname=Path(model_dir, model_name))


## Accuracy with retraining🎯

In [ ]:
X_train = pd.read_parquet(Path(data_dir, "train_set_60.parquet"), engine="fastparquet")
y_train= X_train["buy_sell"]
X_train = X_train[features_classical_size]

X_val= pd.read_parquet(Path(data_dir, "val_set_20.parquet"), engine="fastparquet")
y_val= X_val["buy_sell"]
X_val = X_val[features_classical_size]

X_retrain = pd.concat([X_train, X_val])
del X_train, X_val

y_retrain = pd.concat([y_train, y_val])
del y_train, y_val

In [ ]:
weight = np.geomspace(0.001, 1, num=len(y_retrain))
# keep ordering of data
timestamp = np.linspace(0, 1, len(y_retrain))

# save to pool for faster memory access
retrain_pool = Pool(
            data=X_retrain,
            label=y_retrain,
            cat_features=None,
            weight=weight,
            timestamp=timestamp,
)

model.fit(retrain_pool, verbose=False)

In [ ]:
X_test = pd.read_parquet(Path(data_dir, "test_set_20.parquet"), engine="fastparquet")
y_test = X_test["buy_sell"]
X_test = X_test[features_classical_size]

In [ ]:
acc = model.score(X_test, y_test)
print(acc)

In [ ]:
# without retraining 0.7232624886732101